<a href="https://colab.research.google.com/github/ashpakshaikh26732/Unet-FCN/blob/main/Unet_FCN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**all packages**

In [ ]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 99.5 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.18.0
    Uninstalling tensorflow-2.18.0:
      Successfully uninstalled tensorflow-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 w

In [2]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from IPython.display import display,HTML

In [1]:
!pip install tensorflow==2.18.0
!pip install tensorflow-tpu==2.18.0 --find-links=https://storage.googleapis.com/libtpu-tf-releases/index.html

import tensorflow as tf

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
    print("TPU is running:", tpu.master())
except ValueError as e:
    print("TPU is not avaible:", e)

Looking in links: https://storage.googleapis.com/libtpu-tf-releases/index.html
TPU is running: 


In [3]:
print(f'Number of TPUs: {strategy.num_replicas_in_sync}')

Number of TPUs: 8


In [ ]:
import tensorflow as tf


print(f'TensorFlow version: {tf.__version__}')

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
  print(f'TPU cluster resolved: {tpu.cluster_spec()}')
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)
  print(f'Number of TPUs: {strategy.num_replicas_in_sync}')
except Exception as e:
  print(f'TPU failed to initialize: {e}')

TensorFlow version: 2.18.0
TPU failed to initialize: Please provide a TPU Name to connect to.


**mixed precision training**

In [3]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [4]:
tf.config.run_functions_eagerly(False)


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**copy datasets from drive**

In [6]:
!cp /content/drive/MyDrive/Cityscapes/gtFine_trainvaltest.zip /content/
!cp /content/drive/MyDrive/Cityscapes/leftImg8bit_trainvaltest.zip /content

**extract trainig the data**

In [7]:
os.system("unzip -q /content/gtFine_trainvaltest.zip -d /content/ ")
os.system("unzip -q /content/leftImg8bit_trainvaltest.zip -d /content/")

256

In [8]:
print("Train Images:", len(os.listdir("/content/leftImg8bit/train/")))
print("Train Labels:", len(os.listdir("/content/gtFine/train/")))

Train Images: 18
Train Labels: 18


In [9]:
print("test images : ", len(os.listdir('/content/leftImg8bit/test')))

test images :  6


**Global Valrible**

In [10]:
img_height = 640
img_width = 640
batch_size = 1
final_batch_size = batch_size * strategy.num_replicas_in_sync


**loading images and labels from directry for training**

In [11]:
def load_img():
    train_dir = '/content/leftImg8bit/train'
    train_image_cities = os.listdir(train_dir)
    train_images = []

    for city in sorted(train_image_cities):
        train_city = os.path.join(train_dir, city)
        images = sorted([os.path.join(train_city, img) for img in os.listdir(train_city) if img.endswith('.png')])
        train_images.extend(images)

    return train_images

def load_labels():
    train_dir = '/content/gtFine/train'
    train_labels_cities = os.listdir(train_dir)
    train_labels = []

    for city in sorted(train_labels_cities):
        train_city_label = os.path.join(train_dir, city)
        labels = sorted([f for f in os.listdir(train_city_label) if f.endswith('_gtFine_labelIds.png')])
        for label in labels:
            label_path = os.path.join(train_city_label, label)
            train_labels.append(label_path)

    return train_labels

**reading images from path**

In [12]:
def read_img(img_path):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels=3)
    img = tf.image.resize(img, (img_height, img_width))
    img = img / 255.0
    return img

def read_labels(label_path):
    label = tf.io.read_file(label_path)
    label = tf.io.decode_png(label, channels=1)
    label = tf.image.resize(label, (img_height, img_width), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return tf.cast(label, tf.uint8)

**augmentation**

In [13]:
def process_data(image_path, label_path):
    image = read_img(image_path)
    label = read_labels(label_path)
    return image, label

def augment(image, label):
    label = tf.cast(label, tf.float32)

    if tf.random.uniform(()) > 0.5:
        image = tf.image.flip_left_right(image)
        label = tf.image.flip_left_right(label)

    image = tf.image.random_brightness(image, max_delta=0.1)

    combined = tf.concat([image, label], axis=-1)
    combined = tf.image.random_crop(combined, size=[640, 640, 4])
    image, label = tf.split(combined, num_or_size_splits=[3, 1], axis=-1)

    label = tf.cast(label, tf.uint8)
    return image, label

**creating dataset**

In [14]:
images = load_img()
labels = load_labels()
print("Total Train Images:", len(images))
print("Total Train Labels:", len(labels))
# Create TF Dataset
dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset = dataset.map(process_data, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.map(augment, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=1000).cache().batch(final_batch_size)
train_dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
train_dataset = strategy.experimental_distribute_dataset(train_dataset)

Total Train Images: 2975
Total Train Labels: 2975


In [15]:
for image , label in dataset.take(1):
  print(f'image shape : {image.shape} , label shape {label.shape}')
  print(f'image datatype : {image.dtype}, label dtype {label.dtype}, TYPE : ')

image shape : (8, 640, 640, 3) , label shape (8, 640, 640, 1)
image datatype : <dtype: 'float32'>, label dtype <dtype: 'uint8'>, TYPE : 


**loading images and labels from directry for validiation**

In [16]:
def load_img_val():
    train_dir = '/content/leftImg8bit/val'
    train_image_cities = os.listdir(train_dir)
    train_images = []

    for city in sorted(train_image_cities):
        train_city = os.path.join(train_dir, city)
        images = sorted([os.path.join(train_city, img) for img in os.listdir(train_city) if img.endswith('.png')])
        train_images.extend(images)

    return train_images

def load_labels_val():
    train_dir = '/content/gtFine/val'
    train_labels_cities = os.listdir(train_dir)
    train_labels = []

    for city in sorted(train_labels_cities):
        train_city_label = os.path.join(train_dir, city)
        labels = sorted([f for f in os.listdir(train_city_label) if f.endswith('_gtFine_labelIds.png')])
        for label in labels:
            label_path = os.path.join(train_city_label, label)
            train_labels.append(label_path)

    return train_labels

In [17]:
def read_img_val(img_path):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels=3)
    img = tf.image.resize(img, (img_height, img_width))
    img = img / 255.0
    return img

def read_labels_val(label_path):
    label = tf.io.read_file(label_path)
    label = tf.io.decode_png(label, channels=1)
    label = tf.image.resize(label, (img_height, img_width), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return tf.cast(label, tf.uint8)

In [18]:
def process_data_val(image_path, label_path):
    image = read_img(image_path)
    label = read_labels(label_path)
    return image, label

images = load_img_val()
labels = load_labels_val()

# Create TF Dataset
dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset = dataset.map(process_data_val, num_parallel_calls=tf.data.AUTOTUNE)

dataset = dataset.shuffle(buffer_size=1000).cache().batch(final_batch_size)
val_dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = strategy.experimental_distribute_dataset(val_dataset)

**loading images and labels from directry for testing**

In [19]:
def load_img_test():
    train_dir = '/content/leftImg8bit/test'
    train_image_cities = os.listdir(train_dir)
    train_images = []

    for city in sorted(train_image_cities):
        train_city = os.path.join(train_dir, city)
        images = sorted([os.path.join(train_city, img) for img in os.listdir(train_city) if img.endswith('.png')])
        train_images.extend(images)

    return train_images

def load_labels_test():
    train_dir = '/content/gtFine/test'
    train_labels_cities = os.listdir(train_dir)
    train_labels = []

    for city in sorted(train_labels_cities):
        train_city_label = os.path.join(train_dir, city)
        labels = sorted([f for f in os.listdir(train_city_label) if f.endswith('_gtFine_labelIds.png')])
        for label in labels:
            label_path = os.path.join(train_city_label, label)
            train_labels.append(label_path)

    return train_labels

In [20]:
def read_img_test(img_path):
    img = tf.io.read_file(img_path)
    img = tf.io.decode_png(img, channels=3)
    img = tf.image.resize(img, (img_height, img_width))
    img = img / 255.0
    return img

def read_labels_test(label_path):
    label = tf.io.read_file(label_path)
    label = tf.io.decode_png(label, channels=1)
    label = tf.image.resize(label, (img_height, img_width), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return tf.cast(label, tf.uint8)

In [21]:
def process_data_test(image_path, label_path):
    image = read_img(image_path)
    label = read_labels(label_path)
    return image, label

images = load_img_test()
labels = load_labels_test()

# Create TF Dataset
dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset = dataset.map(process_data_test, num_parallel_calls=tf.data.AUTOTUNE)

dataset = dataset.shuffle(buffer_size=1000).cache().batch(final_batch_size)
test_dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = strategy.experimental_distribute_dataset(test_dataset)

**FCN model**

In [22]:
class Block(tf.keras.models.Model):
    def __init__(self, n_conv, filters, kernel_size, activation, pool_size, pool_stride, block_name):
        super().__init__()
        self.conv_layers = [
            tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel_size, padding='same',
                                   activation=activation, name=f"{block_name}_conv{i+1}")
            for i in range(n_conv)
        ]
        self.pool = tf.keras.layers.MaxPool2D(pool_size=pool_size, strides=pool_stride, name=f"{block_name}_pool")

    def call(self, inputs):
        x = inputs
        for conv in self.conv_layers:
            x = conv(x)
        x = self.pool(x)
        return x

In [23]:
class Encoder(tf.keras.models.Model):
  def __init__(self):
    super().__init__()
    self.block1 = Block(n_conv=2 , filters=64 , kernel_size = (3,3) , activation ='relu' , pool_size=(2,2) , pool_stride = (2,2) , block_name = 'block1')
    self.block2 = Block(n_conv=3 , filters = 128 , kernel_size = (3,3) , activation='relu' , pool_size = (2,2) , pool_stride = (2,2) , block_name = 'block2')
    self.block3 = Block(n_conv=3 , filters = 256 , kernel_size = (3,3) , activation='relu' , pool_size = (2,2) , pool_stride = (2,2) , block_name = 'block3')
    self.block4 = Block(n_conv=3 , filters = 512 , kernel_size = (3,3) , activation='relu' , pool_size = (2,2) , pool_stride = (2,2) , block_name = 'block4')
    self.block5 = Block(n_conv=3 , filters = 512 , kernel_size = (3,3) , activation='relu' , pool_size = (2,2) , pool_stride = (2,2) , block_name = 'block5')
  def call(self,inputs):
    p1 = self.block1(inputs)
    p2 = self.block2(p1)
    p3 = self.block3(p2)
    p4 = self.block4(p3)
    p5 = self.block5(p4)
    return p1 , p2 , p3 , p4 , p5

In [24]:
class Decoder(tf.keras.models.Model):
    def __init__(self, nclasses=19):
        super().__init__()

        self.upsample1 = tf.keras.layers.Conv2DTranspose(nclasses, kernel_size=(4, 4), strides=(2, 2), padding='same', use_bias=False)
        self.upsample2 = tf.keras.layers.Conv2DTranspose(nclasses, kernel_size=(4, 4), strides=(2, 2), padding='same', use_bias=False)
        self.upsample3 = tf.keras.layers.Conv2DTranspose(nclasses, kernel_size=(4, 4), strides=(2, 2), padding='same', use_bias=False)

        self.skip1 = tf.keras.layers.Conv2D(nclasses, kernel_size=(1, 1), activation='relu', padding='same')
        self.skip2 = tf.keras.layers.Conv2D(nclasses, kernel_size=(1, 1), activation='relu', padding='same')
        self.skip3 = tf.keras.layers.Conv2D(nclasses, kernel_size=(1, 1), activation='relu', padding='same')

        self.add1 = tf.keras.layers.Add()
        self.add2 = tf.keras.layers.Add()
        self.add3 = tf.keras.layers.Add()

        self.fcn4 = tf.keras.layers.Conv2DTranspose(nclasses, kernel_size=(4, 4), strides=(4, 4), padding='same', name='fcn4')

        self.softmax = tf.keras.layers.Softmax(axis=-1)

    def call(self, inputs):
        p1, p2, p3, p4, p5 = inputs

        o = self.upsample1(p5)
        o = self.add1([o, self.skip1(p4)])

        o = self.upsample2(o)
        o = self.add2([o, self.skip2(p3)])

        o = self.upsample3(o)
        o = self.add3([o, self.skip3(p2)])

        fcn4 = self.fcn4(o)

        return self.softmax(fcn4)

In [25]:
class FCN(tf.keras.models.Model):
    def __init__(self):
        super().__init__()
        self.Encoder = Encoder()
        self.Decoder = Decoder()

    def call(self, inputs):
        x = self.Encoder(inputs)
        x = self.Decoder(x)
        return x


In [26]:
with strategy.scope():
  fcn =FCN()

**UNET**

In [27]:
class ConvBlock(tf.keras.models.Model):
  def __init__(self, n_filters,block_name , kernel_size=3 , strides = (1,1) , activation='relu'):
    super().__init__()
    self.conv = [tf.keras.layers.Conv2D(filters=n_filters , strides=strides , kernel_size=(kernel_size, kernel_size) , activation='relu', name=f"{block_name}_conv{i+1}", padding='same') for i in range (2)]
  def call(self, inputs):
    x = inputs
    for layer in self.conv:
      x = layer(x)
    return x

In [28]:
class EncoderBlock(tf.keras.models.Model):
  def __init__(self, n_filters , pool_size , dropout,block_name):
    super().__init__()
    self.convBlock = ConvBlock(n_filters=n_filters , block_name=block_name, strides=(1,1))
    self.pool = tf.keras.layers.MaxPooling2D(pool_size=pool_size)
    self.dropout = tf.keras.layers.Dropout(rate = dropout)
  def call (self, inputs) :
    f = self.convBlock(inputs)
    p = self.pool(f)
    p = self.dropout(p)
    return f , p

In [29]:
class Encoder (tf.keras.models.Model):
  def __init__(self):
    super().__init__()
    self.encoder1 = EncoderBlock(64 , pool_size=(2,2),dropout=0.3 , block_name = 'block1')
    self.encoder2 = EncoderBlock(128 , pool_size=(2,2),dropout=0.3 , block_name = 'block2')
    self.encoder3 = EncoderBlock(256 , pool_size=(2,2),dropout=0.3 , block_name = 'block3')
    self.encoder4 = EncoderBlock(512, pool_size=(2,2),dropout=0.3 , block_name = 'block4')
  def call(self, inputs):
    f1,p1=self.encoder1(inputs)
    f2,p2 = self.encoder2(p1)
    f3 , p3 = self.encoder3(p2)
    f4,p4 = self.encoder4(p3)
    return p4,(f1,f2,f3,f4)

In [30]:
class BottleNeck(tf.keras.models.Model):
  def __init__(self):
    super().__init__()
    self.convBlock=ConvBlock(1024, block_name = 'bottleneck' )
  def call(self, inputs):
    x = self.convBlock(inputs)
    return x

In [31]:
class DecoderBlock(tf.keras.models.Model):
  def __init__(self, n_filters ,dropout,block_name , kernel_size= (3,3),strides = (2,2)  ):
    super().__init__()
    self.convTranspose = tf.keras.layers.Conv2DTranspose(n_filters, kernel_size = kernel_size , strides=strides , padding = 'same')
    self.concat = tf.keras.layers.Concatenate()
    self.DropOut = tf.keras.layers.Dropout(dropout)
    self.convBlock = ConvBlock(n_filters=n_filters , block_name =block_name  )
  def call(self, inputs , convOutput):
    u=self.convTranspose(inputs)
    c = self.concat([u,convOutput])
    c= self.DropOut(c)
    c = self.convBlock(c)
    return c

In [32]:

class Decoder (tf.keras.models.Model):
  def __init__(self) :
    super().__init__()
    self.Decoder1 = DecoderBlock(n_filters=512 , dropout=0.3 , block_name='Decoder1')
    self.Decoder2 = DecoderBlock(n_filters=256 , dropout=0.3 , block_name='Decoder2')
    self.Decoder3 = DecoderBlock(n_filters=128 , dropout=0.3 , block_name='Decoder3')
    self.Decoder4 = DecoderBlock(n_filters=64 , dropout=0.3 , block_name='Decoder4')
    self.final_conv = tf.keras.layers.Conv2D(19, kernel_size=(1,1), activation='softmax')
  def call(self , inputs , conv):
    f1, f2 , f3, f4 = conv
    c6 = self.Decoder1(inputs, f4 )
    c7 = self.Decoder2(c6,f3)
    c8 = self.Decoder3(c7,f2)
    c9= self.Decoder4(c8, f1)
    outputs = self.final_conv(c9)
    return outputs


In [33]:
class Unet (tf.keras.models.Model):
  def __init__(self):
    super().__init__()
    self.Encoder = Encoder()
    self.BottleNeck = BottleNeck()
    self.Decoder = Decoder()
  def call(self , inputs):
    encoder_output,convs = self.Encoder(inputs)
    bottleNeck_output = self.BottleNeck(encoder_output)
    output = self.Decoder(bottleNeck_output,convs)
    return output

In [34]:
with strategy.scope():
  unet = Unet()

***UNET ++***

In [35]:
class Unet_plus_plus_convolutional_block(tf.keras.models.Model):
  def __init__(self, n_filters , block_name , activation = 'relu',kernel_size=(3,3)  ):
    super().__init__()
    self.conv_layers = [tf.keras.layers.Conv2D(filters=n_filters , kernel_size= kernel_size , name=f"{block_name}_conv{i+1}",padding = 'same', activation='relu') for i in range (2)]
  def call(self, inputs) :
    x = inputs
    for layer in self.conv_layers:
      x=layer(x)
    return x

In [36]:
class Encoder_Block_for_unt_plus_plus(tf.keras.models.Model):
  def __init__(self,  n_filters , pool_size , dropout,block_name):
    super().__init__()
    self.conv = Unet_plus_plus_convolutional_block(n_filters=n_filters, block_name=block_name)
    self.pool = tf.keras.layers.MaxPooling2D(pool_size=pool_size)
    self.dropout = tf.keras.layers.Dropout(rate =dropout)
  def call(self,inputs):
    f=self.conv(inputs)
    p = self.pool(f)
    p = self.dropout(p)
    return f ,  p

In [37]:
class Encoder_for_unt_plus_plus(tf.keras.models.Model):
  def __init__(self):
    super().__init__()
    self.encoder1 = Encoder_Block_for_unt_plus_plus(n_filters=64, pool_size=(2,2) , dropout=0.3, block_name = 'encoder1')
    self.encoder2 = Encoder_Block_for_unt_plus_plus(n_filters=128, pool_size=(2,2) , dropout=0.3, block_name = 'encoder2')
    self.encoder3 = Encoder_Block_for_unt_plus_plus(n_filters=256, pool_size=(2,2) , dropout=0.3, block_name = 'encoder3')
    self.encoder4 = Encoder_Block_for_unt_plus_plus(n_filters=512, pool_size=(2,2) , dropout=0.3, block_name = 'encoder4')
  def call(self, inputs):
    f1,p1 = self.encoder1(inputs)
    f2,p2 = self.encoder2(p1)
    f3,p3 = self.encoder3(p2)
    f4,p4 = self.encoder4(p3)
    return (f1,f2,f3,f4),(p1,p2,p3,p4)

In [38]:
class BottleNeck_for_unet_plus_plus(tf.keras.models.Model):
  def __init__(self ):
    super().__init__()
    self.bottle_neck = Unet_plus_plus_convolutional_block(n_filters=1024 , block_name = 'bottle_neck'  )
  def call(self, inputs):
    x = self.bottle_neck(inputs)
    return x

In [39]:
class Decoder_Block_for_unet_plus_plus(tf.keras.models.Model):
  def __init__(self,n_filters  , dropout,block_name):
    super().__init__()
    self.conv = Unet_plus_plus_convolutional_block(n_filters=n_filters , block_name = block_name)
    self.dropout = tf.keras.layers.Dropout(rate = dropout)
    self.convTranspose = tf.keras.layers.Conv2DTranspose(n_filters, kernel_size = (3,3) , strides=(2,2) , padding = 'same')
    self.concat = tf.keras.layers.Concatenate()

  def call(self, conv_output_conc, inputs):




      if inputs.shape[1] <= 1 or inputs.shape[2] <= 1:

          target_size = (max(conv_output_conc.shape[1], 1), max(conv_output_conc.shape[2], 1))
          u = tf.image.resize(inputs, target_size)
      else:
          u = self.convTranspose(inputs)


      if u.shape[1:3] != conv_output_conc.shape[1:3]:

          if u.shape[1] * u.shape[2] < conv_output_conc.shape[1] * conv_output_conc.shape[2]:
              u = tf.image.resize(u, (conv_output_conc.shape[1], conv_output_conc.shape[2]))
          else:
              conv_output_conc = tf.image.resize(conv_output_conc, (u.shape[1], u.shape[2]))

      c = self.concat([u, conv_output_conc])
      c = self.dropout(c)
      c = self.conv(c)
      return c

In [40]:
class Decoder_for_unet_plus_plus(tf.keras.models.Model):
  def __init__(self,num_classes = 19):
    super().__init__()
    self.decoder11 = Decoder_Block_for_unet_plus_plus(n_filters=64  , dropout = 0.3, block_name = 'skip_conv_block_11')
    self.decoder12 = Decoder_Block_for_unet_plus_plus(n_filters=64  , dropout = 0.3, block_name = 'skip_conv_block_12')
    self.decoder13 = Decoder_Block_for_unet_plus_plus(n_filters=64  , dropout = 0.3, block_name = 'skip_conv_block_13')
    self.decoder14 = Decoder_Block_for_unet_plus_plus(n_filters=64  , dropout = 0.3, block_name = 'skip_conv_block_14')
    self.decoder21 = Decoder_Block_for_unet_plus_plus(n_filters=128  , dropout = 0.3, block_name = 'skip_conv_block_21')
    self.decoder22 = Decoder_Block_for_unet_plus_plus(n_filters=128  , dropout = 0.3, block_name = 'skip_conv_block_22')
    self.decoder23 = Decoder_Block_for_unet_plus_plus(n_filters=128  , dropout = 0.3, block_name = 'skip_conv_block_23')
    self.decoder31 = Decoder_Block_for_unet_plus_plus(n_filters=256  , dropout = 0.3, block_name = 'skip_conv_block_31')
    self.decoder32 = Decoder_Block_for_unet_plus_plus(n_filters=256  , dropout = 0.3, block_name = 'skip_conv_block_32')
    self.decoder41 = Decoder_Block_for_unet_plus_plus(n_filters=512  , dropout = 0.3, block_name = 'skip_conv_block_41')

    self.concat12=tf.keras.layers.Concatenate()
    self.concat13=tf.keras.layers.Concatenate()
    self.concat14=tf.keras.layers.Concatenate()

    self.concat22=tf.keras.layers.Concatenate()
    self.concat23=tf.keras.layers.Concatenate()

    self.concat32=tf.keras.layers.Concatenate()
    self.bottle_neck = BottleNeck_for_unet_plus_plus()
    self.output_layer14 = tf.keras.layers.Conv2D(
        filters=num_classes,
        kernel_size=(1,1),
        activation='sigmoid' if num_classes == 1 else 'softmax',
        padding='same',
        name='output_layer'
        )
    self.output_layer13 = tf.keras.layers.Conv2D(
        filters=num_classes,
        kernel_size=(1,1),
        activation='sigmoid' if num_classes == 1 else 'softmax',
        padding='same',
        name='output_layer'
        )
    self.output_layer12 = tf.keras.layers.Conv2D(
        filters=num_classes,
        kernel_size=(1,1),
        activation='sigmoid' if num_classes == 1 else 'softmax',
        padding='same',
        name='output_layer'
        )
    self.output_layer11 = tf.keras.layers.Conv2D(
        filters=num_classes,
        kernel_size=(1,1),
        activation='sigmoid' if num_classes == 1 else 'softmax',
        padding='same',
        name='output_layer'
        )

  def call(self, convs_cont , pool_cont):
    f1,f2,f3,f4 = convs_cont
    p1,p2,p3,p4 = pool_cont
    for i, feat in enumerate([f1, f2, f3, f4, p1, p2, p3, p4]):
        if feat.shape[1] < 1 or feat.shape[2] < 1:
            print(f"Warning: Feature map {i} has invalid dimensions: {feat.shape}")
    p5 = self.bottle_neck(p4)
    decoder41 = self.decoder41(f4,p5)
    decoder31 = self.decoder31(f3,p4)
    conc32 = self.concat32([f3 , decoder31])
    decoder32 = self.decoder32(conc32 , decoder41)
    decoder21 = self.decoder21(f2,p3)
    conc22 = self.concat22([f2,decoder21])
    decoder22 = self.decoder22(conc22 , decoder31)
    conc23 = self.concat23([f2,decoder21, decoder22])
    decoder23 = self.decoder23(conc23 , decoder32)
    decoder11 = self.decoder11(f1,p2)
    conc12 = self.concat12([f1,decoder11])
    decoder12 = self.decoder12(conc12 , decoder21)
    conc13 = self.concat13([f1,decoder11,decoder12])
    decoder13 = self.decoder13(conc13 , decoder22)
    conc14 = self.concat14([f1 , decoder11  , decoder12 , decoder13])
    decoder14 = self.decoder14(conc14 , decoder23)
    output14 = self.output_layer14(decoder14)
    output13 = self.output_layer13(decoder13)
    output12 = self.output_layer12(decoder12)
    output11 = self.output_layer11(decoder11)
    return output11,output12,output13,output14

In [41]:
class Unet_plus_plus(tf.keras.models.Model):
    def __init__(self, min_input_size=16):
        super().__init__()
        self.min_input_size = min_input_size
        self.encoder = Encoder_for_unt_plus_plus()
        self.decoder = Decoder_for_unet_plus_plus()

    def call(self, inputs):

        input_shape = inputs.shape
        if input_shape[1] < self.min_input_size or input_shape[2] < self.min_input_size:
            raise ValueError(f"Input image dimensions must be at least {self.min_input_size}x{self.min_input_size}")

        y, z = self.encoder(inputs)
        output11, output12, output13, output14 = self.decoder(y, z)
        return output11, output12, output13, output14

In [42]:
with strategy.scope():
  unet_plus_plus = Unet_plus_plus()

**testing code for model**

**defining loss**

In [43]:
with strategy.scope():

  class DiceLoss(tf.keras.losses.Loss):
      def __init__(self, smooth=1e-6):
          super().__init__()
          self.smooth = smooth

      def call(self, y_true, y_pred):
          y_true = tf.squeeze(y_true, axis=-1)
          y_true_f = tf.keras.backend.flatten(tf.one_hot(tf.cast(y_true, tf.int32), depth=tf.shape(y_pred)[-1]))
          y_pred_f = tf.keras.backend.flatten(y_pred)

          intersection = tf.reduce_sum(y_true_f * y_pred_f)
          denominator = tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f)

          dice_loss = 1 - (2.0 * intersection + self.smooth) / (denominator + self.smooth)
          return dice_loss

  class SemanticSegmentationLoss(tf.keras.losses.Loss):
      def __init__(self, alpha=1.0, beta=1.0):
          super().__init__()
          self.dice_loss = DiceLoss()
          self.alpha = alpha
          self.beta = beta

      def call(self, y_true, y_pred):
          ce = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)
          dice_loss = self.dice_loss(y_true, y_pred)
          return self.alpha * ce + self.beta * dice_loss


In [44]:
with strategy.scope():
  class DeepSupervisionLoss(tf.keras.losses.Loss):
      def __init__(self, weights=None, smooth=1e-6):
          super().__init__()
          self.smooth = smooth
          self.weights = weights or [1.0]
          self.ce_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

      def dice_loss(self, y_true, y_pred):
          y_true = tf.squeeze(y_true, axis=-1)
          num_classes = tf.shape(y_pred)[-1]
          y_true_one_hot = tf.one_hot(tf.cast(y_true, tf.int32), depth=num_classes)
          intersection = tf.reduce_sum(y_true_one_hot * y_pred, axis=[1, 2])
          denominator = tf.reduce_sum(y_true_one_hot, axis=[1, 2]) + tf.reduce_sum(y_pred, axis=[1, 2])
          return 1 - (2.0 * intersection + self.smooth) / (denominator + self.smooth)

      def call(self, y_true, y_pred):
          total_loss = 0.0
          num_outputs = len(y_pred)
          self.weights = self.weights[:num_outputs] + [1.0] * (num_outputs - len(self.weights))
          for i in range(num_outputs):
              ce = self.ce_loss(y_true, y_pred[i])
              dice = self.dice_loss(y_true, y_pred[i])
              total_loss += self.weights[i] * (ce + dice)
          return total_loss


**IOU metric**

In [45]:
class IOUMetric(tf.keras.metrics.Metric):
    def __init__(self, n_classes=19, smooth=1e-6, name="iou", **kwargs):
        super().__init__(name=name, **kwargs)
        self.n_classes = n_classes
        self.smooth = smooth
        self.iou_sum = self.add_weight(name="iou_sum", initializer="zeros")
        self.count = self.add_weight(name="count", initializer="zeros")
    @tf.function
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.squeeze(y_true, axis=-1)
        y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=self.n_classes)
        # y_pred = tf.nn.softmax(y_pred, axis=-1)
        y_pred = tf.cast(y_pred, dtype=tf.float32)

        intersection = tf.reduce_sum(y_true * y_pred, axis=[0, 1, 2])
        union = tf.reduce_sum(y_true, axis=[0, 1, 2]) + tf.reduce_sum(y_pred, axis=[0, 1, 2]) - intersection


        iou_per_class = (intersection + self.smooth) / (union + self.smooth)


        valid_classes = tf.cast(union > 0, tf.float32)
        num_valid_classes = tf.reduce_sum(valid_classes)


        mean_iou = tf.reduce_sum(iou_per_class * valid_classes) / tf.maximum(num_valid_classes, 1.0)


        self.iou_sum.assign_add(mean_iou)
        self.count.assign_add(1)

    def result(self):
        return self.iou_sum / tf.maximum(self.count, 1.0)

    def reset_state(self):
        self.iou_sum.assign(0)
        self.count.assign(0)

    def get_config(self):
        config = super().get_config()
        config.update({"n_classes": self.n_classes, "smooth": self.smooth})
        return config


**per class IOU metrics**

In [46]:
class PerClassIOUMetric(tf.keras.metrics.Metric):
    def __init__(self, n_classes=19, smooth=1e-6, name="iou_per_class", **kwargs):
        super().__init__(name=name, **kwargs)
        self.n_classes = n_classes
        self.smooth = smooth
        self.sum_union = self.add_weight(name="sum_union", shape=(n_classes,), initializer="zeros")
        self.sum_intersection = self.add_weight(name="sum_intersection", shape=(n_classes,), initializer="zeros")
    @tf.function
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.squeeze(y_true, axis=-1)
        y_true = tf.one_hot(tf.cast(y_true, dtype=tf.int32), depth=self.n_classes)
        y_pred = tf.cast(y_pred , dtype = tf.float32 )

        intersection = tf.reduce_sum(y_true * y_pred, axis=[0, 1, 2])
        union = tf.reduce_sum(y_true, axis=[0, 1, 2]) + tf.reduce_sum(y_pred, axis=[0, 1, 2]) - intersection


        self.sum_union.assign_add(union)
        self.sum_intersection.assign_add(intersection)
    @tf.function
    def result(self):
        return (self.sum_intersection + self.smooth) / (self.sum_union + self.smooth)

    def reset_state(self):
        self.sum_intersection.assign(tf.zeros_like(self.sum_intersection))
        self.sum_union.assign(tf.zeros_like(self.sum_union))


**Dice Coeficient Metrics**

In [47]:
class Dice_Coefficent_matrics(tf.keras.metrics.Metric):
    def __init__(self, n_classes=19, smooth=1e-6, name='dice_coefficient', **kwargs):
        super().__init__(name=name, **kwargs)
        self.n_classes = n_classes
        self.smooth = smooth

        self.sum_dice_coefficent = self.add_weight(name="sum_dice_coefficent", initializer="zeros", dtype=tf.float32)
        self.count = self.add_weight(name="count", initializer="zeros", dtype=tf.float32)


    @tf.function
    def update_state(self, y_true, y_pred, sample_weight=None):

        y_true = tf.squeeze(y_true, axis=-1)
        y_pred = tf.cast(y_pred, dtype=tf.float32)
        y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=self.n_classes)


        intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2])
        total_area = tf.reduce_sum(y_true, axis=[1, 2]) + tf.reduce_sum(y_pred, axis=[1, 2])


        dice_score = 2 * (intersection + self.smooth) / (total_area + self.smooth)


        dice_score = tf.reduce_mean(dice_score, axis=-1)


        mean_dice = tf.reduce_mean(dice_score)


        self.sum_dice_coefficent.assign_add(mean_dice)
        self.count.assign_add(1.0)


    def result(self):
        return self.sum_dice_coefficent / tf.maximum(self.count, 1.0)

    def reset_state(self):
        self.sum_dice_coefficent.assign(0.0)
        self.count.assign(0.0)


**per class_dice_coefficent metrics**

In [48]:
class PerClassDiceCoefficientMatrics(tf.keras.metrics.Metric):
    def __init__(self, n_classes=19, smooth=1e-6, name="dice_per_class", **kwargs):
        super().__init__(name=name, **kwargs)
        self.n_classes = n_classes
        self.smooth = smooth
        self.sum_intersection = self.add_weight(name="sum_intersection", shape=(n_classes,), initializer="zeros")
        self.sum_union = self.add_weight(name="sum_union", shape=(n_classes,), initializer="zeros")
    @tf.function
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.squeeze(y_true, axis=-1)
        y_true = tf.one_hot(tf.cast(y_true, dtype=tf.int32), depth=self.n_classes)
        y_pred = tf.cast(y_pred, dtype=tf.float32)
        intersection = tf.reduce_sum(y_true * y_pred, axis=[0, 1, 2])
        sum_areas = tf.reduce_sum(y_true, axis=[0, 1, 2]) + tf.reduce_sum(y_pred, axis=[0, 1, 2])

        self.sum_intersection.assign_add(intersection)
        self.sum_union.assign_add(sum_areas)

    def result(self):
        return (2.0 * self.sum_intersection + self.smooth) / (self.sum_union + self.smooth)

    def reset_state(self):
        self.sum_intersection.assign(tf.zeros_like(self.sum_intersection))
        self.sum_union.assign(tf.zeros_like(self.sum_union))


**Pixel Accurcy Metrics**

In [49]:
class Pixel_accurcy_metrics(tf.keras.metrics.Metric):
  def __init__(self, n_classes=19, name="pixel_accuracy", **kwargs):
    super().__init__(name=name, **kwargs)
    self.n_classes = n_classes
    self.sum_pixel_wise_accurcy = self.add_weight(name="sum_correct_pixels", initializer="zeros", dtype=tf.float32)
    self.count = self.add_weight(name="total_pixels", initializer="zeros", dtype=tf.float32)

  @tf.function
  def update_state(self, y_true, y_pred, sample_weight=None):
    y_true = tf.squeeze(y_true, axis=-1)
    y_pred = tf.cast(y_pred, dtype=tf.float32)
    y_true = tf.one_hot(tf.cast(y_true, tf.int32), depth=self.n_classes)
    y_pred = tf.one_hot(tf.argmax(y_pred, axis=-1), depth=self.n_classes)

    correct_pixel = tf.reduce_sum(y_true * y_pred)
    total_pixel = tf.reduce_prod(tf.shape(y_true)[:-1])


    pixel_accuracy = correct_pixel / tf.maximum(tf.cast(total_pixel, tf.float32), 1.0)
    self.sum_pixel_wise_accurcy.assign_add(pixel_accuracy)
    self.count.assign_add(1)


  def result(self):
    return self.sum_pixel_wise_accurcy / tf.maximum(self.count, 1.0)

  def reset_state(self):
    self.sum_pixel_wise_accurcy.assign(0)
    self.count.assign(0)


**optimizer**

In [50]:
with strategy.scope():
    optimizer_fcn = mixed_precision.LossScaleOptimizer(
        tf.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-5),
        dynamic=True
    )
    optimizer_unet = mixed_precision.LossScaleOptimizer(
        tf.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-5),
        dynamic=True
    )
    optimizer_unet_plus_plus = mixed_precision.LossScaleOptimizer(
        tf.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-5),
        dynamic=True
    )

    # Explicitly build optimizer states


**checkPoint dirs**

In [51]:
checkpoint_dir_fcn = '/content/drive/MyDrive/CityScapes_model_checkpoints/fcn'
checkpoint_dir_unet = '/content/drive/MyDrive/CityScapes_model_checkpoints/unet'
checkpoint_dir_unet_plus_plus = '/content/drive/MyDrive/CityScapes_model_checkpoints/unet_plus_plus'
log_dir = '/content/drive/MyDrive/CityScapes_model_checkpoints/log_dir'

**custom callbacks**

In [52]:
class Custom_checkpoint_Callabck(tf.keras.callbacks.Callback):
  def __init__(self, checkpoint_dir_fcn , checkpoint_dir_unet , checkpoint_dir_unet_plus_plus , fcn , unet , unet_plus_plus , optimizer_fcn ,optimizer_unet ,optimizer_unet_plus_plus  ) :
    super().__init__()
    self.checkpoint_dir_fcn = checkpoint_dir_fcn
    self.checkpoint_dir_unet = checkpoint_dir_unet
    self.checkpoint_dir_unet_plus_plus = checkpoint_dir_unet_plus_plus
    self.checkpoint_fcn = tf.train.Checkpoint(
        model = fcn ,
        optimizer = optimizer_fcn,
        epoch = tf.Variable(0 , dtype = tf.int64)
    )
    self.checkpoint_manager_for_fcn = tf.train.CheckpointManager(
        self.checkpoint_fcn ,
        self.checkpoint_dir_fcn ,
        max_to_keep=3
    )
    self.checkpoint_unet = tf.train.Checkpoint(
        model = unet ,
        optimizer = optimizer_unet,
        epoch = tf.Variable(0 , dtype = tf.int64)
    )
    self.checkpoint_manager_for_unet = tf.train.CheckpointManager(
        self.checkpoint_unet ,
        self.checkpoint_dir_unet ,
        max_to_keep=3
    )
    self.checkpoint_unet_plus_plus = tf.train.Checkpoint(
        model = unet_plus_plus ,
        optimizer = optimizer_unet_plus_plus,
        epoch = tf.Variable(0 , dtype = tf.int64)
    )
    self.checkpoint_manager_for_unet_plus_plus = tf.train.CheckpointManager(
        self.checkpoint_unet_plus_plus ,
        self.checkpoint_dir_unet_plus_plus ,
        max_to_keep=3
    )
  def on_epoch_end(self , epoch, logs = None):
    self.checkpoint_fcn.epoch.assign(epoch+1)
    self.checkpoint_manager_for_fcn.save()
    print(f"\nCheckpoint saved for epoch {epoch } at {self.checkpoint_manager_for_fcn.latest_checkpoint} for fcn model")
    self.checkpoint_unet.epoch.assign(epoch+1)
    self.checkpoint_manager_for_unet.save()
    print(f"\nCheckpoint saved for epoch {epoch } at {self.checkpoint_manager_for_unet.latest_checkpoint} for unet model")
    self.checkpoint_unet_plus_plus.epoch.assign(epoch+1)
    self.checkpoint_manager_for_unet_plus_plus.save()
    print(f"\nCheckpoint saved for epoch {epoch } at {self.checkpoint_manager_for_unet_plus_plus.latest_checkpoint} for unet_plus_plus model")
  def load_latest_model(self):
    if self.checkpoint_manager_for_fcn.latest_checkpoint and self.checkpoint_manager_for_unet.latest_checkpoint and self.checkpoint_manager_for_unet_plus_plus.latest_checkpoint :

      self.checkpoint_fcn.restore(self.checkpoint_manager_for_fcn.latest_checkpoint)
      self.checkpoint_unet.restore(self.checkpoint_manager_for_unet.latest_checkpoint)
      self.checkpoint_unet_plus_plus.restore(self.checkpoint_manager_for_unet_plus_plus.latest_checkpoint)
      start_epoch =self.checkpoint_fcn.epoch.numpy()
      print(f"Restored from checkpoint: {self.checkpoint_manager_for_fcn.latest_checkpoint}, "
                  f"resuming from epoch {start_epoch}")
      print(f"Restored from checkpoint: {self.checkpoint_manager_for_unet.latest_checkpoint}, "
                  f"resuming from epoch {start_epoch}")
      print(f"Restored from checkpoint: {self.checkpoint_manager_for_unet_plus_plus.latest_checkpoint}, "
                  f"resuming from epoch {start_epoch}")
      return start_epoch
    else :
      print('no checkpoint found starting from epoch 1')
      return 1


In [53]:
class Custom_early_stopping_callback(tf.keras.callbacks.Callback):
  def __init__(self, paticance = 15  , min_delta = 1e-4):
    self.patiance = paticance
    self.min_delta = min_delta
    self.best_loss = {
        'fcn' : float('inf') ,
        'unet' : float('inf') ,
        'unet_plus_plus' : float('inf')
    }
    self.stoped_epoch = 0
    self.wait = {
        'fcn' : 0 ,
        'unet': 0 ,
        'unet_plus_plus' : 0
    }
  def early_stoping(self, epoch , val_loss):
    if (val_loss['fcn']< self.best_loss['fcn']-self.min_delta):
      self.best_loss['fcn'] = val_loss['fcn']
      self.wait['fcn'] = 0
    else :
      self.wait['fcn'] +=1
    if (val_loss['unet']< self.best_loss['unet']-self.min_delta):
      self.best_loss['unet'] = val_loss['unet']
      self.wait['unet'] = 0
    else  :
      self.wait['unet']+=1
    if (val_loss['unet_plus_plus']< self.best_loss['unet_plus_plus']-self.min_delta):
      self.best_loss['unet_plus_plus'] = val_loss['unet_plus_plus']
      self.wait['unet_plus_plus'] = 0
    else :
      self.wait['unet_plus_plus']+=1
    if self.wait['fcn'] > self.patiance and self.wait['unet']>self.patiance and self.wait['unet_plus_plus']>self.patiance:
      self.stoped_epoch  =epoch
      print (f'early stoping trainig at {self.stoped_epoch}')
      return True
    return False


In [54]:
class Custom_learning_rate_schedule(tf.keras.callbacks.Callback):
  def __init__(self , optimizer_fcn , optimizer_unet , optimizer_unet_plus_plus , factor = 0.5 , patiance = 5 , min_delta = 1e-4 , min_lr = 1e-8 ):
    super().__init__()
    self.factor = factor
    self.patiance = patiance
    self.min_delta = min_delta
    self.min_lr = min_lr
    self.best_val_loss = {
        'fcn' : float('inf') ,
        'unet': float('inf') ,
        'unet_plus_plus' : float('inf')
    }
    self.optimzer  = {
        'fcn' : optimizer_fcn ,
        'unet' : optimizer_fcn ,
        'unet_plus_plus': optimizer_unet_plus_plus
    }
    self.wait = {
        'fcn' : 0 ,
        'unet': 0 ,
        'unet_plus_plus' : 0
    }
  def change_learning_rate(self, epoch , val_loss):
    for model_name , val_loss in val_loss.items():
      if val_loss < self.best_val_loss[model_name]-self.min_delta:
        self.best_val_loss[model_name] = val_loss
        self.wait[model_name] = 0
      else :
        self.wait[model_name] +=1
      if self.wait[model_name] >= self.patiance :
        old_lr = self.optimzer[model_name].lr.numpy()
        new_lr = tf.maximum(old_lr*self.factor , self.min_lr)
        self.optimzer[model_name].lr.assign(new_lr)
        print (f'learning rate for model : {model_name} has been change from : {old_lr} to the new lr : {new_lr} at epoch : {epoch}')
        self.wait= 0


In [55]:
class Custom_Training_logger_callback(tf.keras.callbacks.Callback):
  def __init__(self, batches_per_epoch):
    super().__init__()
    self.model_name = ['fcn' , 'unet' , 'unet_plus_plus']
    self.batch_per_epoch = batches_per_epoch
    self.batch_table_display = None
    self.graph_display = None
  def On_epoch_begin(self, epoch ):
    print(f'Epoch begins : {epoch}')
    self.progress_bar = tqdm(total=self.batch_per_epoch,desc=f'epoch {epoch}')
    self.batch_table_data = []
    self.batch_loss = {
        'fcn':[] ,
        'unet': [] ,
        'unet_plus_plus' :[]
    }

    self.epoch_table_data = []
    display(HTML('<h3>📊 Batch-wise Loss Table</h3>'))


  def on_batch_end(self, batch , epoch,data = None):
    self.progress_bar.set_postfix({
        f'epoch :{epoch} , batch ': batch / self.batch_per_epoch
        })
    self.progress_bar.update(1)
    if data == None :
      return
    self.batch_table_data.append({
        'batch': batch ,
        'fcn_loss' : data['fcn']['loss'] ,

        'unet_loss' :data['unet']['loss'],

        'unet_plus_plus_loss' : data['unet_plus_plus']['loss']  ,

        'fcn_IOUMetric' : data['fcn']['metrics']['IOUMetric'] ,
        'fcn_PerClassIOUMetric' : data['fcn']['metrics']['PerClassIOUMetric'],
        'fcn_Dice_Coefficent_matrics' :  data['fcn']['metrics']['Dice_Coefficent_matrics'],
        'fcn_PerClassDiceCoefficientMatrics' : data['fcn']['metrics']['PerClassDiceCoefficientMatrics'],
        'fcn_Pixel_accurcy_metrics' : data['fcn']['metrics']['Pixel_accurcy_metrics'],
        'unet_IOUMetric' : data['unet']['metrics']['IOUMetric'] ,
        'unet_PerClassIOUMetric' : data['unet']['metrics']['PerClassIOUMetric'],
        'unet_Dice_Coefficent_matrics' :  data['unet']['metrics']['Dice_Coefficent_matrics'],
        'unet_PerClassDiceCoefficientMatrics' : data['unet']['metrics']['PerClassDiceCoefficientMatrics'],
        'unet_Pixel_accurcy_metrics' : data['unet']['metrics']['Pixel_accurcy_metrics'],
        'unet_plus_plus_IOUMetric' : data['unet_plus_plus']['metrics']['IOUMetric'] ,
        'unet_plus_plus_PerClassIOUMetric' : data['unet_plus_plus']['metrics']['PerClassIOUMetric'],
        'unet_plus_plus_Dice_Coefficent_matrics' :  data['unet_plus_plus']['metrics']['Dice_Coefficent_matrics'],
        'unet_plus_plus_PerClassDiceCoefficientMatrics' : data['unet_plus_plus']['metrics']['PerClassDiceCoefficientMatrics'],
        'unet_plus_plus_Pixel_accurcy_metrics' : data['unet_plus_plus']['metrics']['Pixel_accurcy_metrics'],

      })


    df = pd.DataFrame(self.batch_table_data)

    if self.batch_table_display is None:
      self.batch_table_display = display( df, display_id=True)
    else :
      self.batch_table_display.update(df)
    self.batch_loss['fcn'].append(data['fcn']['loss'])
    self.batch_loss['unet'].append(data['unet']['loss'])
    self.batch_loss['unet_plus_plus'].append(data['unet_plus_plus']['loss'])

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    axes[0].plot(self.batch_loss['fcn'], label='FCN Loss', marker='o')
    axes[0].plot(self.batch_loss['unet'], label='U-Net Loss', marker='o')
    axes[0].plot(self.batch_loss['unet_plus_plus'], label='U-Net++ Loss', marker='o')
    axes[0].set_title('Training Loss')
    axes[0].set_xlabel('Batch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True)



    plt.tight_layout()
    if self.graph_display is None:
      display(HTML('<h3>📉 Loss Graphs (Batch-wise Trend)</h3>'))
      self.graph_display = display(fig, display_id=True)
    else:
      self.graph_display.update(fig)
    plt.close(fig)


  def on_epoch_end(self , epoch ,data):
    self.progress_bar.close()
    if data == None :
      return
    display(HTML('<h3>📊 Epoch Summary Table</h3>'))
    self.epoch_table_data.append({
        'epoch': epoch ,
        'fcn_loss' : data['fcn']['loss'] ,
        'fcn_val_loss':data['fcn']['val_loss'] ,
        'unet_loss' :data['unet']['loss'],
        'unet_val_loss' : data['unet']['val_loss'] ,
        'unet_plus_plus_loss' : data['unet_plus_plus']['loss']  ,
        'unet_plus_plus_val_loss' : data['unet_plus_plus']['val_loss'],
        'fcn_IOUMetric' : data['fcn']['metrics']['IOUMetric'] ,
        'fcn_PerClassIOUMetric' : data['fcn']['metrics']['PerClassIOUMetric'],
        'fcn_Dice_Coefficent_matrics' :  data['fcn']['metrics']['Dice_Coefficent_matrics'],
        'fcn_PerClassDiceCoefficientMatrics' : data['fcn']['metrics']['PerClassDiceCoefficientMatrics'],
        'fcn_Pixel_accurcy_metrics' : data['fcn']['metrics']['Pixel_accurcy_metrics'],
        'unet_IOUMetric' : data['unet']['metrics']['IOUMetric'] ,
        'unet_PerClassIOUMetric' : data['unet']['metrics']['PerClassIOUMetric'],
        'unet_Dice_Coefficent_matrics' :  data['unet']['metrics']['Dice_Coefficent_matrics'],
        'unet_PerClassDiceCoefficientMatrics' : data['unet']['metrics']['PerClassDiceCoefficientMatrics'],
        'unet_Pixel_accurcy_metrics' : data['unet']['metrics']['Pixel_accurcy_metrics'],
        'unet_plus_plus_IOUMetric' : data['unet_plus_plus']['metrics']['IOUMetric'] ,
        'unet_plus_plus_PerClassIOUMetric' : data['unet_plus_plus']['metrics']['PerClassIOUMetric'],
        'unet_plus_plus_Dice_Coefficent_matrics' :  data['unet_plus_plus']['metrics']['Dice_Coefficent_matrics'],
        'unet_plus_plus_PerClassDiceCoefficientMatrics' : data['unet_plus_plus']['metrics']['PerClassDiceCoefficientMatrics'],
        'unet_plus_plus_Pixel_accurcy_metrics' : data['unet_plus_plus']['metrics']['Pixel_accurcy_metrics'],

        'fcn_IOUMetric_val' : data['fcn']['metrics']['IOUMetric_val'] ,
        'fcn_PerClassIOUMetric_val' : data['fcn']['metrics']['PerClassIOUMetric_val'],
        'fcn_Dice_Coefficent_matrics_val' :  data['fcn']['metrics']['Dice_Coefficent_matrics_val'],
        'fcn_PerClassDiceCoefficientMatrics_val' : data['fcn']['metrics']['PerClassDiceCoefficientMatrics_val'],
        'fcn_Pixel_accurcy_metrics_val' : data['fcn']['metrics']['Pixel_accurcy_metrics_val'],
        'unet_IOUMetric_val' : data['unet']['metrics']['IOUMetric_val'] ,
        'unet_PerClassIOUMetric_val' : data['unet']['metrics']['PerClassIOUMetric_val'],
        'unet_Dice_Coefficent_matrics_val' :  data['unet']['metrics']['Dice_Coefficent_matrics_val'],
        'unet_PerClassDiceCoefficientMatrics_val' : data['unet']['metrics']['PerClassDiceCoefficientMatrics_val'],
        'unet_Pixel_accurcy_metrics_val' : data['unet']['metrics']['Pixel_accurcy_metrics_val'],
        'unet_plus_plus_IOUMetric_val' : data['unet_plus_plus']['metrics']['IOUMetric_val'] ,
        'unet_plus_plus_PerClassIOUMetric_val' : data['unet_plus_plus']['metrics']['PerClassIOUMetric_val'],
        'unet_plus_plus_Dice_Coefficent_matrics_val' :  data['unet_plus_plus']['metrics']['Dice_Coefficent_matrics_val'],
        'unet_plus_plus_PerClassDiceCoefficientMatrics_val' : data['unet_plus_plus']['metrics']['PerClassDiceCoefficientMatrics_val'],
        'unet_plus_plus_Pixel_accurcy_metrics_val' : data['unet_plus_plus']['metrics']['Pixel_accurcy_metrics_val'],
      })
    df=pd.DataFrame(self.epoch_table_data)
    display(df)
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    axes[0].plot(self.batch_loss['fcn'], label='FCN Loss', marker='o')
    axes[0].plot(self.batch_loss['unet'], label='U-Net Loss', marker='o')
    axes[0].plot(self.batch_loss['unet_plus_plus'], label='U-Net++ Loss', marker='o')
    axes[0].set_title('Training Loss')
    axes[0].set_xlabel('Batch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True)




    plt.tight_layout()
    display(HTML(f'<h3>📉 Final Loss Graphs - Epoch {epoch + 1}</h3>'))
    self.final_graph_display = display(fig, display_id=f'final_graph_epoch_{epoch + 1}')
    plt.close(fig)

In [56]:
class Master_callback(tf.keras.callbacks.Callback):
  def __init__(self, log_dir , checkpoint_dir_fcn , checkpoint_dir_unet ,                 checkpoint_dir_unet_plus_plus , fcn , unet , unet_plus_plus , optimizer_fcn ,optimizer_unet ,   optimizer_unet_plus_plus , batches_per_epoch  ):
    self.writer=tf.summary.create_file_writer(logdir=log_dir)
    # initilizing custom checkpoint callback
    self.ccc = Custom_checkpoint_Callabck(checkpoint_dir_fcn=checkpoint_dir_fcn,checkpoint_dir_unet=checkpoint_dir_unet , checkpoint_dir_unet_plus_plus=checkpoint_dir_unet_plus_plus , fcn = fcn , unet = unet , unet_plus_plus=unet_plus_plus , optimizer_fcn=optimizer_fcn , optimizer_unet=optimizer_unet , optimizer_unet_plus_plus=optimizer_unet_plus_plus )
    # initilizing custom ealry stoping callback
    self.cesc = Custom_early_stopping_callback()
    # initilizing custom learning rate scheduler
    self.clrs = Custom_learning_rate_schedule(optimizer_fcn=optimizer_fcn , optimizer_unet=optimizer_unet , optimizer_unet_plus_plus=optimizer_unet_plus_plus)
    # initilizing custom training logger
    self.ctlc = Custom_Training_logger_callback(batches_per_epoch=batches_per_epoch)
    self.stop = False

  def on_train_begin(self):
    return self.ccc.load_latest_model()
  def on_epoch_begin(self, epoch ):
    self.ctlc.On_epoch_begin(epoch)
  def on_batch_end(self,batch , epoch , data):
    self.ctlc.on_batch_end(batch , epoch , data)
  def on_epoch_end(self, epoch , data):
    self.ctlc.on_epoch_end(epoch , data)
    # summary -->
    with self.writer.as_default():
      tf.summary.scalar('fcn_loss' , data['fcn']['loss'],step=epoch)
      tf.summary.scalar('fcn_val_loss' , data['fcn']['val_loss'],step=epoch)
      tf.summary.scalar('unet_loss' , data['unet']['loss'],step=epoch)
      tf.summary.scalar('uent_val_loss' , data['unet']['val_loss'],step=epoch)
      tf.summary.scalar('unet_plus_plus_loss' , data['unet_plus_plus']['loss'],step=epoch)
      tf.summary.scalar('uent_plus_plus_val_loss' , data['unet_plus_plus']['val_loss'],step=epoch)
      tf.summary.scalar('fcn_IOUMetric',data['fcn']['metrics']['IOUMetric'] , step = epoch)
      tf.summary.scalar('fcn_PerClassIOUMetric' , data['fcn']['metrics']['PerClassIOUMetric'] , step = epoch)
      tf.summary.scalar('fcn_Dice_Coefficent_matrics' ,  data['fcn']['metrics']['Dice_Coefficent_matrics'] , step = epoch)
      tf.summary.scalar('fcn_PerClassDiceCoefficientMatrics' , data['fcn']['metrics']['PerClassDiceCoefficientMatrics'] , step = epoch)
      tf.summary.scalar('fcn_Pixel_accurcy_metrics' , data['fcn']['metrics']['Pixel_accurcy_metrics'], step = epoch)

      tf.summary.scalar('unet_IOUMetric',data['unet']['metrics']['IOUMetric'] , step = epoch)
      tf.summary.scalar('unet_PerClassIOUMetric' , data['unet']['metrics']['PerClassIOUMetric'] , step = epoch)
      tf.summary.scalar('unet_Dice_Coefficent_matrics' ,  data['unet']['metrics']['Dice_Coefficent_matrics'] , step = epoch)
      tf.summary.scalar('unet_PerClassDiceCoefficientMatrics' , data['unet']['metrics']['PerClassDiceCoefficientMatrics'] , step = epoch)
      tf.summary.scalar('unet_Pixel_accurcy_metrics' , data['unet']['metrics']['Pixel_accurcy_metrics'], step = epoch)

      tf.summary.scalar('unet_plus_plus_IOUMetric',data['unet_plus_plus']['metrics']['IOUMetric'] , step = epoch)
      tf.summary.scalar('unet_plus_plus_PerClassIOUMetric' , data['unet_plus_plus']['metrics']['PerClassIOUMetric'] , step = epoch)
      tf.summary.scalar('unet_plus_plus_Dice_Coefficent_matrics' ,  data['unet_plus_plus']['metrics']['Dice_Coefficent_matrics'] , step = epoch)
      tf.summary.scalar('unet_plus_plus_PerClassDiceCoefficientMatrics' , data['unet_plus_plus']['metrics']['PerClassDiceCoefficientMatrics'] , step = epoch)
      tf.summary.scalar('unet_plus_plus_Pixel_accurcy_metrics' , data['unet_plus_plus']['metrics']['Pixel_accurcy_metrics'], step = epoch)

      tf.summary.scalar('fcn_IOUMetric_val',data['fcn']['metrics']['IOUMetric_val'] , step = epoch)
      tf.summary.scalar('fcn_PerClassIOUMetric_val' , data['fcn']['metrics']['PerClassIOUMetric_val'] , step = epoch)
      tf.summary.scalar('fcn_Dice_Coefficent_matrics_val' ,  data['fcn']['metrics']['Dice_Coefficent_matrics_val'] , step = epoch)
      tf.summary.scalar('fcn_PerClassDiceCoefficientMatrics_val' , data['fcn']['metrics']['PerClassDiceCoefficientMatrics_val'] , step = epoch)
      tf.summary.scalar('fcn_Pixel_accurcy_metrics_val' , data['fcn']['metrics']['Pixel_accurcy_metrics_val'], step = epoch)

      tf.summary.scalar('unet_IOUMetric_val',data['unet']['metrics']['IOUMetric_val'] , step = epoch)
      tf.summary.scalar('unet_PerClassIOUMetric_val' , data['unet']['metrics']['PerClassIOUMetric_val'] , step = epoch)
      tf.summary.scalar('unet_Dice_Coefficent_matrics_val' ,  data['unet']['metrics']['Dice_Coefficent_matrics_val'] , step = epoch)
      tf.summary.scalar('unet_PerClassDiceCoefficientMatrics_val' , data['unet']['metrics']['PerClassDiceCoefficientMatrics_val'] , step = epoch)
      tf.summary.scalar('unet_Pixel_accurcy_metrics_val' , data['unet']['metrics']['Pixel_accurcy_metrics__val'], step = epoch)

      tf.summary.scalar('unet_plus_plus_IOUMetric_val',data['unet_plus_plus']['metrics']['IOUMetric_val'] , step = epoch)
      tf.summary.scalar('unet_plus_plus_PerClassIOUMetric_val' , data['unet_plus_plus']['metrics']['PerClassIOUMetric_val'] , step = epoch)
      tf.summary.scalar('unet_plus_plus_Dice_Coefficent_matrics_val' ,  data['unet_plus_plus']['metrics']['Dice_Coefficent_matrics_val'] , step = epoch)
      tf.summary.scalar('unet_plus_plus_PerClassDiceCoefficientMatrics_val' , data['unet_plus_plus']['metrics']['PerClassDiceCoefficientMatrics_val'] , step = epoch)
      tf.summary.scalar('unet_plus_plus_Pixel_accurcy_metrics_val' , data['unet_plus_plus']['metrics']['Pixel_accurcy_metrics_val'], step = epoch)
    self.writer.flush()
    self.ccc.on_epoch_end(epoch)
    self.clrs.change_learning_rate(epoch ,{
        'fcn' : data['fcn']['val_loss'] ,
        'unet' :  data['unet']['val_loss'] ,
        'unet_plus_plus' :  data['unet_plus_plus']['val_loss']
    })
    return self.cesc.early_stoping(epoch , {
        'fcn' : data['fcn']['val_loss'] ,
        'unet' :  data['unet']['val_loss'] ,
        'unet_plus_plus' :  data['unet_plus_plus']['val_loss']
    })

**initilizing instances**

In [57]:
with strategy.scope():
  fcn_SemanticSegmentationLoss=SemanticSegmentationLoss()
  unet_SemanticSegmentationLoss=SemanticSegmentationLoss()
  deepSupervisionLoss=DeepSupervisionLoss(weights = [0.5, 0.3, 0.15, 0.05])

In [63]:
batches_per_epoch = 372
master_callback =Master_callback(log_dir=log_dir , checkpoint_dir_fcn=checkpoint_dir_fcn , checkpoint_dir_unet=checkpoint_dir_unet , checkpoint_dir_unet_plus_plus=checkpoint_dir_unet_plus_plus , fcn=fcn , unet = unet , unet_plus_plus=unet_plus_plus ,batches_per_epoch=batches_per_epoch ,optimizer_fcn=optimizer_fcn , optimizer_unet=optimizer_unet , optimizer_unet_plus_plus=optimizer_unet_plus_plus )



# fcn matrics
with strategy.scope() :
  fcn_IOUMetric = IOUMetric()
  fcn_PerClassIOUMetric = PerClassIOUMetric()
  fcn_Dice_Coefficent_matrics = Dice_Coefficent_matrics()
  fcn_PerClassDiceCoefficientMatrics = PerClassDiceCoefficientMatrics()
  fcn_Pixel_accurcy_metrics  = Pixel_accurcy_metrics()

  fcn_IOUMetric_val = IOUMetric()
  fcn_PerClassIOUMetric_val = PerClassIOUMetric()
  fcn_Dice_Coefficent_matrics_val = Dice_Coefficent_matrics()
  fcn_PerClassDiceCoefficientMatrics_val = PerClassDiceCoefficientMatrics()
  fcn_Pixel_accurcy_metrics_val  = Pixel_accurcy_metrics()

  # unet matrics
  unet_IOUMetric = IOUMetric()
  unet_PerClassIOUMetric = PerClassIOUMetric()
  unet_Dice_Coefficent_matrics = Dice_Coefficent_matrics()
  unet_PerClassDiceCoefficientMatrics = PerClassDiceCoefficientMatrics()
  unet_Pixel_accurcy_metrics  = Pixel_accurcy_metrics()

  unet_IOUMetric_val = IOUMetric()
  unet_PerClassIOUMetric_val = PerClassIOUMetric()
  unet_Dice_Coefficent_matrics_val = Dice_Coefficent_matrics()
  unet_PerClassDiceCoefficientMatrics_val = PerClassDiceCoefficientMatrics()
  unet_Pixel_accurcy_metrics_val  = Pixel_accurcy_metrics()

  # unet_plus_plus matrics
  unet_plus_plus_IOUMetric = IOUMetric()
  unet_plus_plus_PerClassIOUMetric = PerClassIOUMetric()
  unet_plus_plus_Dice_Coefficent_matrics = Dice_Coefficent_matrics()
  unet_plus_plus_PerClassDiceCoefficientMatrics = PerClassDiceCoefficientMatrics()
  unet_plus_plus_Pixel_accurcy_metrics  = Pixel_accurcy_metrics()

  unet_plus_plus_IOUMetric_val = IOUMetric()
  unet_plus_plus_PerClassIOUMetric_val = PerClassIOUMetric()
  unet_plus_plus_Dice_Coefficent_matrics_val = Dice_Coefficent_matrics()
  unet_plus_plus_PerClassDiceCoefficientMatrics_val = PerClassDiceCoefficientMatrics()
  unet_plus_plus_Pixel_accurcy_metrics_val  = Pixel_accurcy_metrics()

**custom training**

In [64]:
with strategy.scope():
  @tf.function
  def train_step(x_train_batch, y_train_batch):
    with tf.GradientTape(persistent=True) as tape:
      # tf.print("Input shapes - x:", x_train_batch.shape, "y:", y_train_batch.shape)
      fcn_logits = fcn(x_train_batch)
      # tf.print("fcn_logits shape:", fcn_logits.shape)
      fcn_loss = fcn_SemanticSegmentationLoss(y_true=y_train_batch, y_pred=fcn_logits)
      # tf.print("fcn_loss computed:", fcn_loss)
      unet_logits = unet(x_train_batch)
      # tf.print("unet_logits shape:", unet_logits.shape)
      unet_loss = unet_SemanticSegmentationLoss(y_true=y_train_batch, y_pred=unet_logits)
      # tf.print("unet_loss computed:", unet_loss)
      unet_plus_plus_outputs = unet_plus_plus(x_train_batch)
      unet_plus_plus_loss = deepSupervisionLoss(y_true=y_train_batch, y_pred=unet_plus_plus_outputs)
      unet_plus_plus_logits = unet_plus_plus_outputs[-1]
      # tf.print("unet_plus_plus_logits shape:", unet_plus_plus_logits.shape)

    fcn_gradients = tape.gradient(fcn_loss, fcn.trainable_variables)
    # tf.print("fcn_gradients computed, length:", len(fcn_gradients))
    unet_gradients = tape.gradient(unet_loss, unet.trainable_variables)
    # tf.print("unet_gradients computed, length:", len(unet_gradients))
    unet_plus_plus_gradients = tape.gradient(unet_plus_plus_loss, unet_plus_plus.trainable_variables)
    # tf.print("unet_plus_plus_gradients computed, length:", len(unet_plus_plus_gradients))



    optimizer_fcn.apply_gradients(zip(fcn_gradients,fcn.trainable_variables))
    optimizer_unet.apply_gradients(zip(unet_gradients,unet.trainable_variables))
    optimizer_unet_plus_plus.apply_gradients(zip(unet_plus_plus_gradients , unet_plus_plus.trainable_variables))

    fcn_IOUMetric.update_state(y_true=y_train_batch, y_pred=fcn_logits)
    fcn_PerClassIOUMetric.update_state(y_true=y_train_batch, y_pred=fcn_logits)
    fcn_Dice_Coefficent_matrics.update_state(y_train_batch, fcn_logits)
    fcn_PerClassDiceCoefficientMatrics.update_state(y_train_batch, fcn_logits)
    fcn_Pixel_accurcy_metrics.update_state(y_train_batch, fcn_logits)

    unet_IOUMetric.update_state(y_true=y_train_batch, y_pred=unet_logits)
    unet_PerClassIOUMetric.update_state(y_true=y_train_batch, y_pred=unet_logits)
    unet_Dice_Coefficent_matrics.update_state(y_train_batch, unet_logits)
    unet_PerClassDiceCoefficientMatrics.update_state(y_train_batch, unet_logits)
    unet_Pixel_accurcy_metrics.update_state(y_train_batch, unet_logits)

    unet_plus_plus_IOUMetric.update_state(y_true=y_train_batch, y_pred=unet_plus_plus_logits)
    unet_plus_plus_PerClassIOUMetric.update_state(y_true=y_train_batch, y_pred=unet_plus_plus_logits)
    unet_plus_plus_Dice_Coefficent_matrics.update_state(y_train_batch, unet_plus_plus_logits)
    unet_plus_plus_PerClassDiceCoefficientMatrics.update_state(y_train_batch, unet_plus_plus_logits)
    unet_plus_plus_Pixel_accurcy_metrics.update_state(y_train_batch, unet_plus_plus_logits)

    del tape
    return {
      'fcn_loss': fcn_loss,
      'unet_loss': unet_loss,
      'unet_plus_plus_loss' : unet_plus_plus_loss

    }


In [65]:
with strategy.scope():
  @tf.function
  def val_step(x_val_batch,y_val_batch):
    fcn_logits = fcn(x_val_batch)
    fcn_val_loss = fcn_SemanticSegmentationLoss(y_true=y_val_batch,y_pred=fcn_logits)
    unet_logits  = unet(x_val_batch)
    unet_val_loss = unet_SemanticSegmentationLoss(y_true = y_val_batch,y_pred = unet_logits)
    unet_plus_plus_logits = unet_plus_plus(x_val_batch)
    unet_plus_plus_val_loss = deepSupervisionLoss(y_true = y_val_batch, y_pred = unet_plus_plus_logits)

    fcn_IOUMetric_val.update_state(y_true=y_val_batch, y_pred=fcn_logits)
    fcn_PerClassIOUMetric_val.update_state(y_true=y_val_batch, y_pred=fcn_logits)
    fcn_Dice_Coefficent_matrics_val.update_state(y_val_batch, fcn_logits)
    fcn_PerClassDiceCoefficientMatrics_val.update_state(y_val_batch, fcn_logits)
    fcn_Pixel_accurcy_metrics_val.update_state(y_val_batch, fcn_logits)

    unet_IOUMetric_val.update_state(y_true=y_val_batch, y_pred=unet_logits)
    unet_PerClassIOUMetric_val.update_state(y_true=y_val_batch, y_pred=unet_logits)
    unet_Dice_Coefficent_matrics_val.update_state(y_val_batch, unet_logits)
    unet_PerClassDiceCoefficientMatrics_val.update_state(y_val_batch, unet_logits)
    unet_Pixel_accurcy_metrics_val.update_state(y_val_batch, unet_logits)

    unet_plus_plus_IOUMetric_val.update_state(y_true=y_val_batch, y_pred=unet_plus_plus_logits)
    unet_plus_plus_PerClassIOUMetric_val.update_state(y_true=y_val_batch, y_pred=unet_plus_plus_logits)
    unet_plus_plus_Dice_Coefficent_matrics_val.update_state(y_val_batch, unet_plus_plus_logits)
    unet_plus_plus_PerClassDiceCoefficientMatrics_val.update_state(y_val_batch, unet_plus_plus_logits)
    unet_plus_plus_Pixel_accurcy_metrics_val.update_state(y_val_batch, unet_plus_plus_logits)
    return {
        'fcn_val_loss' : fcn_val_loss ,
        'unet_val_loss' : unet_val_loss ,
        'unet_plus_plus_val_loss' : unet_plus_plus_val_loss
    }

In [66]:
with strategy.scope() :
  @tf.function
  def distributed_train_step(x_train_batch , y_train_batch):
    per_replica_results = strategy.run(train_step, args=(x_train_batch, y_train_batch))




    fcn_loss = strategy.reduce(tf.distribute.ReduceOp.MEAN,per_replica_results['fcn_loss'],axis=None)
    unet_loss = strategy.reduce(tf.distribute.ReduceOp.MEAN,per_replica_results['unet_loss'],axis=None)
    unet_plus_plus_loss = strategy.reduce(tf.distribute.ReduceOp.MEAN,per_replica_results['unet_plus_plus_loss'],axis = None)

    return {
        'fcn' : fcn_loss ,
        'unet' : unet_loss ,
        'unet_plus_plus' : unet_plus_plus_loss
    }

In [67]:
with strategy.scope() :
  @tf.function
  def distributed_val_step(x_val_batch,y_val_batch):
    per_replica_results = strategy.run(val_step , args = (x_val_batch,y_val_batch))
    fcn_val_loss = strategy.reduce(tf.distribute.ReduceOp.MEAN,per_replica_results['fcn_val_loss'] , axis = None)
    unet_val_loss = strategy.reduce(tf.distribute.ReduceOp.MEAN,per_replica_results['unet_val_loss'] , axis = None)
    unet_plus_plus_val_loss = strategy.reduce(tf.distribute.ReduceOp.MEAN,per_replica_results['unet_plus_plus_val_loss'] , axis = None)
    return {
        'fcn' : fcn_val_loss ,
        'unet' : unet_val_loss ,
        'unet_plus_plus' : unet_plus_plus_val_loss
    }

In [68]:
with strategy.scope():

  def train_model_for_one_epoch(epoch):
    fcn_loss = []
    unet_loss = []
    unet_plus_plus_loss = []
    for step  , (x_train_batch , y_train_batch) in enumerate (train_dataset):
      losses=distributed_train_step(x_train_batch , y_train_batch)
      fcn_loss.append(losses['fcn'])
      unet_loss.append(losses['unet'])
      unet_plus_plus_loss.append(losses['unet_plus_plus'])
      data = {
          'fcn': {
              'loss' : losses['fcn'] ,
              'metrics': {
                  'IOUMetric' : fcn_IOUMetric.result(),
                  'PerClassIOUMetric' : fcn_PerClassIOUMetric.result(),
                  'Dice_Coefficent_matrics' : fcn_Dice_Coefficent_matrics.result(),
                  'PerClassDiceCoefficientMatrics' : fcn_PerClassDiceCoefficientMatrics.result() ,
                  'Pixel_accurcy_metrics' : fcn_Pixel_accurcy_metrics.result()
              }
          },
          'unet' : {
              'loss' : losses['unet'] ,
              'metrics': {
                  'IOUMetric' : unet_IOUMetric.result(),
                  'PerClassIOUMetric' : unet_PerClassIOUMetric.result(),
                  'Dice_Coefficent_matrics' : unet_Dice_Coefficent_matrics.result(),
                  'PerClassDiceCoefficientMatrics' : unet_PerClassDiceCoefficientMatrics.result() ,
                  'Pixel_accurcy_metrics' : unet_Pixel_accurcy_metrics.result()
              }
          },
          'unet_plus_plus' : {
              'loss' : losses['unet_plus_plus'] ,
              'metrics': {
                  'IOUMetric' : unet_plus_plus_IOUMetric.result(),
                  'PerClassIOUMetric' : unet_plus_plus_PerClassIOUMetric.result(),
                  'Dice_Coefficent_matrics' : unet_plus_plus_Dice_Coefficent_matrics.result(),
                  'PerClassDiceCoefficientMatrics' : unet_plus_plus_PerClassDiceCoefficientMatrics.result() ,
                  'Pixel_accurcy_metrics' : unet_plus_plus_Pixel_accurcy_metrics.result()
              }
          }
      }
      master_callback.on_batch_end(batch = step+1 , epoch = epoch,data = data)
    return {
        'fcn' : fcn_loss ,
        'unet' : unet_loss  ,
        'unet_plus_plus' : unet_plus_plus_loss
    }

In [69]:
with strategy.scope() :
  @tf.function
  def val_model_for_one_epoch():
    fcn_val_loss = []
    unet_val_loss = []
    unet_plus_plus_val_loss = []
    for step , (x_val_batch,y_val_batch) in enumerate(val_dataset):
      losses = distributed_val_step(x_val_batch , y_val_batch)
      fcn_val_loss.append(losses['fcn'])
      unet_val_loss.append(losses['unet'])
      unet_plus_plus_val_loss.append(losses['unet_plus_plus'])
    return {
        'fcn' : fcn_val_loss ,
        'unet' : unet_val_loss ,
        'unet_plus_plus' : unet_plus_plus_val_loss
    }

In [ ]:
with strategy.scope():
  stop_training =False
  start=master_callback.on_train_begin()
  for epoch in range(start , 250 ):
    master_callback.on_epoch_begin(epoch)
    losses = train_model_for_one_epoch(epoch)
    val_losses = val_model_for_one_epoch()
    data = {
            'fcn': {
                'loss' : losses['fcn'] ,
                'val_loss' : val_losses['fcn'] ,
                'metrics': {
                    'IOUMetric' : fcn_IOUMetric.result(),
                    'PerClassIOUMetric' : fcn_PerClassIOUMetric.result(),
                    'Dice_Coefficent_matrics' : fcn_Dice_Coefficent_matrics.result(),
                    'PerClassDiceCoefficientMatrics' : fcn_PerClassDiceCoefficientMatrics.result() ,
                    'Pixel_accurcy_metrics' : fcn_Pixel_accurcy_metrics.result(),

                    'IOUMetric_val' : fcn_IOUMetric_val.result(),
                    'PerClassIOUMetric_val' : fcn_PerClassIOUMetric_val.result(),
                    'Dice_Coefficent_matrics_val' : fcn_Dice_Coefficent_matrics_val.result(),
                    'PerClassDiceCoefficientMatrics_val' : fcn_PerClassDiceCoefficientMatrics_val.result() ,
                    'Pixel_accurcy_metrics_val' : fcn_Pixel_accurcy_metrics_val.result()
                }
            },
            'unet' : {
                'loss' : losses['unet'] ,
                'val_loss' : val_losses['unet'],
                'metrics': {
                    'IOUMetric' : unet_IOUMetric.result(),
                    'PerClassIOUMetric' : unet_PerClassIOUMetric.result(),
                    'Dice_Coefficent_matrics' : unet_Dice_Coefficent_matrics.result(),
                    'PerClassDiceCoefficientMatrics' : unet_PerClassDiceCoefficientMatrics.result() ,
                    'Pixel_accurcy_metrics' : unet_Pixel_accurcy_metrics.result(),

                    'IOUMetric_val' : unet_IOUMetric_val.result(),
                    'PerClassIOUMetric_val' : unet_PerClassIOUMetric_val.result(),
                    'Dice_Coefficent_matrics_val' : unet_Dice_Coefficent_matrics_val.result(),
                    'PerClassDiceCoefficientMatrics_val' : unet_PerClassDiceCoefficientMatrics_val.result() ,
                    'Pixel_accurcy_metrics_val' : unet_Pixel_accurcy_metrics_val.result()
                }
            },
            'unet_plus_plus' : {
                'loss' : losses['unet_plus_plus'] ,
                'val_loss' : val_losses['unet_plus_plus'] ,
                'metrics': {
                    'IOUMetric' : unet_plus_plus_IOUMetric.result(),
                    'PerClassIOUMetric' : unet_plus_plus_PerClassIOUMetric.result(),
                    'Dice_Coefficent_matrics' : unet_plus_plus_Dice_Coefficent_matrics.result(),
                    'PerClassDiceCoefficientMatrics' : unet_plus_plus_PerClassDiceCoefficientMatrics.result() ,
                    'Pixel_accurcy_metrics' : unet_plus_plus_Pixel_accurcy_metrics.result(),

                    'IOUMetric_val' : unet_plus_plus_IOUMetric_val.result(),
                    'PerClassIOUMetric_val' : unet_plus_plus_PerClassIOUMetric_val.result(),
                    'Dice_Coefficent_matrics_val' : unet_plus_plus_Dice_Coefficent_matrics_val.result(),
                    'PerClassDiceCoefficientMatrics_val' : unet_plus_plus_PerClassDiceCoefficientMatrics_val.result() ,
                    'Pixel_accurcy_metrics_val' : unet_plus_plus_Pixel_accurcy_metrics_val.result()
                }
            }
        }
    fcn_IOUMetric.reset_state()
    fcn_IOUMetric_val.reset_state()
    fcn_PerClassIOUMetric.reset_state()
    fcn_PerClassIOUMetric_val.reset_state()
    fcn_Dice_Coefficent_matrics.reset_state()
    fcn_Dice_Coefficent_matrics_val.reset_state()
    fcn_PerClassDiceCoefficientMatrics.reset_state()
    fcn_PerClassDiceCoefficientMatrics_val.reset_state()
    fcn_Pixel_accurcy_metrics.reset_state()
    fcn_Pixel_accurcy_metrics_val.reset_state()

    unet_IOUMetric.reset_state()
    unet_IOUMetric_val.reset_state()
    unet_PerClassIOUMetric.reset_state()
    unet_PerClassIOUMetric_val.reset_state()
    unet_Dice_Coefficent_matrics.reset_state()
    unet_Dice_Coefficent_matrics_val.reset_state()
    unet_PerClassDiceCoefficientMatrics.reset_state()
    unet_PerClassDiceCoefficientMatrics_val.reset_state()
    unet_Pixel_accurcy_metrics.reset_state()
    unet_Pixel_accurcy_metrics_val.reset_state()

    unet_plus_plus_IOUMetric.reset_state()
    unet_plus_plus_IOUMetric_val.reset_state()
    unet_plus_plus_PerClassIOUMetric.reset_state()
    unet_plus_plus_PerClassIOUMetric_val.reset_state()
    unet_plus_plus_Dice_Coefficent_matrics.reset_state()
    unet_plus_plus_Dice_Coefficent_matrics_val.reset_state()
    unet_plus_plus_PerClassDiceCoefficientMatrics.reset_state()
    unet_plus_plus_PerClassDiceCoefficientMatrics_val.reset_state()
    unet_plus_plus_Pixel_accurcy_metrics.reset_state()
    unet_plus_plus_Pixel_accurcy_metrics_val.reset_state()

    stop_training=master_callback.on_epoch_end(epoch=epoch , data= data)
    if stop_training :
      break

no checkpoint found starting from epoch 1
Epoch begins : 1


epoch 1:   0%|          | 0/372 [00:00<?, ?it/s]

In [ ]:
with strategy.scope():
  @tf.function
  def train_step(x_batch_train,y_batch_train):
    with tf.GradientTape()  as tape :
      logits=fcn(x_batch_train)
      loss = fcn_SemanticSegmentationLoss(y_batch_train,logits)
    gradients = tape.gradient(loss , fcn.trainable_variables)
    optimizer_fcn.apply_gradients(zip(gradients, fcn.trainable_variables))
    return loss
  @tf.function
  def distributed_train_step(x_train_batch,y_train_batch):
    per_replica_loss =strategy.run(train_step , args = (x_train_batch , y_train_batch))
    return  strategy.reduce(tf.distribute.ReduceOp.MEAN,per_replica_loss,axis=None)
  for epoch in range (2):
    print('started training {epoch}')
    for step , (x_batch_train,y_batch_train ) in enumerate(train_dataset) :
      print(f'starting of step : {step}')
      error=distributed_train_step(x_batch_train,y_batch_train)
      print(error)
      print(f'end of step : {step}')
    print(f'end of epoch {epoch}')
  print('finish training ')

started training {epoch}
starting of step : 0
tf.Tensor(nan, shape=(), dtype=float32)
end of step : 0
starting of step : 1
tf.Tensor(nan, shape=(), dtype=float32)
end of step : 1
starting of step : 2
tf.Tensor(nan, shape=(), dtype=float32)
end of step : 2
starting of step : 3
tf.Tensor(nan, shape=(), dtype=float32)
end of step : 3
starting of step : 4
tf.Tensor(nan, shape=(), dtype=float32)
end of step : 4
starting of step : 5
tf.Tensor(nan, shape=(), dtype=float32)
end of step : 5
starting of step : 6
tf.Tensor(nan, shape=(), dtype=float32)
end of step : 6
starting of step : 7
tf.Tensor(nan, shape=(), dtype=float32)
end of step : 7
starting of step : 8
tf.Tensor(nan, shape=(), dtype=float32)
end of step : 8
starting of step : 9
tf.Tensor(nan, shape=(), dtype=float32)
end of step : 9
starting of step : 10
tf.Tensor(nan, shape=(), dtype=float32)
end of step : 10
starting of step : 11
tf.Tensor(nan, shape=(), dtype=float32)
end of step : 11
starting of step : 12
tf.Tensor(nan, shape=(), 